# Bug / Issue

Boundary conditions that are JIT functions are not correctly handled by the 
saddle point solver. These work for Poisson / Poisson-Vector problems just fine
but something seems to go wrong during the SNES solve for saddle point problems.

Sometimes this seems to be true for sympy functions, sometimes not, but it is 
reproducible if the uw function is a mesh variable of any kind. 

In the example below, this fails for the saddle point system.
```
incompressible_vector_projection.add_dirichlet_bc( (s_soln.fn,0.0), "Bottom" , (1,) )
```
but this works
```
incompressible_vector_projection.add_dirichlet_bc( (0.0,0.0), "Bottom" , (1,) )
```
and apparently, so does a sympy function of x,y that must also be compiled. 

In [1]:
import petsc4py
import underworld3 as uw
import numpy as np

In [2]:
meshbox = uw.util_mesh.UnstructuredSimplexBox(
                                             minCoords=(0.0,0.0), 
                                             maxCoords=(1.0,1.0), 
                                             cellSize=1.0/32.0, 
                                             regular=True)

In [3]:
# Doesn't matter which element type - this one is also broken !
# meshbox = uw.util_mesh.StructuredQuadBox(
#                                            minCoords=(0.0,0.0), 
#                                            maxCoords=(1.0,1.0), 
#                                            elementRes=(32,32)
# )

In [4]:
import sympy

# Some useful coordinate stuff 

x = meshbox.N.x
y = meshbox.N.y
z = meshbox.N.z

In [5]:
# Project values from a swarm to a mesh variable (with bcs)

swarm  = uw.swarm.Swarm(mesh=meshbox)
s_values  = uw.swarm.SwarmVariable("Ss", swarm, 1,           proxy_degree=3)
v_values  = uw.swarm.SwarmVariable("Vs", swarm, meshbox.dim, proxy_degree=3)
iv_values = uw.swarm.SwarmVariable("Vi", swarm, meshbox.dim, proxy_degree=3)

swarm.populate(fill_param=3)

In [6]:
s_soln  = uw.mesh.MeshVariable("T",    meshbox,  1,            degree=2 )
v_soln  = uw.mesh.MeshVariable('U',    meshbox,  meshbox.dim,  degree=2 )
iv_soln = uw.mesh.MeshVariable('IU',   meshbox,  meshbox.dim,  degree=2 )

In [7]:
s_fn = sympy.cos(5.0*sympy.pi * x) * sympy.cos(5.0*sympy.pi * y)
sv_fn = meshbox.vector.gradient(s_fn)

In [8]:
vector_projection = uw.systems.Vector_Projection(meshbox, v_soln)
vector_projection.uw_function = v_values.f
vector_projection.smoothing = 1.0e-3

# Velocity boundary conditions (compare left / right walls in the soln !)

vector_projection.add_dirichlet_bc( (0.0,), "Left" ,   (0,) )
vector_projection.add_dirichlet_bc( (0.0,), "Right" ,  (0,) )
vector_projection.add_dirichlet_bc( (0.0,), "Top" ,    (1,) )
vector_projection.add_dirichlet_bc( (s_soln.fn, ), "Bottom" , (1,) )

In [9]:
# try to enforce incompressibility

incompressible_vector_projection = uw.systems.Solenoidal_Vector_Projection(meshbox, iv_soln)
incompressible_vector_projection.uw_function =  sv_fn
incompressible_vector_projection.smoothing = 1.0e-3  # see how well it works !

# Velocity boundary conditions (compare left / right walls in the soln !)
incompressible_vector_projection.add_dirichlet_bc( (0.0,), "Left" ,   (0,) )
incompressible_vector_projection.add_dirichlet_bc( (0.0,), "Right" ,  (0,) )
incompressible_vector_projection.add_dirichlet_bc( (0.0,), "Top" ,    (1,) )
incompressible_vector_projection.add_dirichlet_bc( (s_soln.fn, ), "Bottom" , (1,) )

In [10]:
with swarm.access(s_values, v_values, iv_values):
    s_values.data[:,0]  = uw.function.evaluate(s_fn, swarm.data)    
    v_values.data[:,0]  = uw.function.evaluate(sympy.cos(5.0*sympy.pi * x) * sympy.cos(5.0*sympy.pi * y), swarm.data)    
    v_values.data[:,1]  = uw.function.evaluate(sympy.sin(5.0*sympy.pi * x) * sympy.sin(5.0*sympy.pi * y), swarm.data)
    iv_values.data[:,0] = uw.function.evaluate(sympy.vector.curl(v_soln.fn).dot(meshbox.N.i), swarm.data)    
    iv_values.data[:,1] = uw.function.evaluate(sympy.vector.curl(v_soln.fn).dot(meshbox.N.j), swarm.data)    

In [11]:
vector_projection.solve()

  0 SNES Function norm 0.0124731 
    Residual norms for VProj1_ solve.
    0 KSP Residual norm 1.247310381572e-02 
    1 KSP Residual norm 9.719335137827e-03 
    2 KSP Residual norm 1.510511014108e-03 
    3 KSP Residual norm 5.868799298067e-04 
    4 KSP Residual norm 2.323208892874e-04 
    5 KSP Residual norm 7.381165585584e-05 
    6 KSP Residual norm 3.025643958251e-05 
    7 KSP Residual norm 1.007938249575e-05 
  1 SNES Function norm 1.00794e-05 
Nonlinear VProj1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


In [ ]:
incompressible_vector_projection.solve()

  0 SNES Function norm 0.196986 
    Residual norms for iVProj1_ solve.
    0 KSP Residual norm 2.992412855221e+01 
